<a href="https://colab.research.google.com/github/AreebAhmad-02/Embedding-Models-Finetuning/blob/main/Fine_tuning_Embedding_Models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## imports and install

In [10]:
!pip install llama_index -q -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 43.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 29.5 MB/s eta 0:00:00


In [11]:
import json

from llama_index.core import VectorStoreIndex, SimpleDirectoryReader,StorageContext,load_index_from_storage
from llama_index.core.node_parser import SimpleNodeParser
from llama_index.core.schema import MetadataMode
from llama_index.core import Settings

In [17]:
TRAIN_CORPUS_FPATH_SENTENCE_SPLIT = './corpus/train_corpus_sentence_split.json'
EVAL_CORPUS_FPATH_SENTENCE_SPLIT = './corpus/eval_corpus_sentence_split.json'


In [14]:
TRAIN_FILES = ["/content/data/batch1preprocessed_file.txt","/content/data/batch2preprocessed_file.txt"]
VAL_FILES = ['/content/data/batch0preprocessed_file.txt']

TRAIN_CORPUS_FPATH = './corpus/train_corpus.json'
VAL_CORPUS_FPATH = './corpus/val_corpus.json'

#Chunking

## simple chunking splitting

In [ ]:
def load_corpus(files, verbose=False):
    if verbose:
        print(f"Loading files {files}")

    reader = SimpleDirectoryReader(input_files=files)
    docs = reader.load_data()
    if verbose:
        print(f'Loaded {len(docs)} docs')

    parser = SimpleNodeParser.from_defaults()
    nodes = parser.get_nodes_from_documents(docs, show_progress=verbose)

    if verbose:
        print(f'Parsed {len(nodes)} nodes')

    corpus = {node.node_id: node.get_content(metadata_mode=MetadataMode.NONE) for node in nodes}
    return corpus

In [ ]:
train_corpus = load_corpus(TRAIN_FILES, verbose=True)
val_corpus = load_corpus(VAL_FILES, verbose=True)

Loading files ['/content/batch1preprocessed_file.txt', '/content/batch2preprocessed_file.txt']
Loaded 2 docs


Parsing nodes:   0%|          | 0/2 [00:00<?, ?it/s]

Parsed 66 nodes
Loading files ['/content/data/preprocessed_file.txt']
Loaded 1 docs


Parsing nodes:   0%|          | 0/1 [00:00<?, ?it/s]

Parsed 30 nodes


In [ ]:
with open(TRAIN_CORPUS_FPATH, 'w+') as f:
    json.dump(train_corpus, f)

with open(VAL_CORPUS_FPATH, 'w+') as f:
    json.dump(val_corpus, f)

## Sentence split chunking using

In [13]:
from llama_index.core.node_parser import SentenceSplitter


def load_corpus_sentence_splitter(files, verbose=False):
    if verbose:
        print(f"Loading files {files}")

    reader = SimpleDirectoryReader(input_files=files)
    docs = reader.load_data()
    if verbose:
        print(f'Loaded {len(docs)} docs')


    splitter = SentenceSplitter(
    chunk_size=524,
    chunk_overlap=20,)
    nodes = splitter.get_nodes_from_documents(docs, show_progress=verbose)

    if verbose:
        print(f'Parsed {len(nodes)} nodes')

    corpus = {node.node_id: node.get_content(metadata_mode=MetadataMode.NONE) for node in nodes}
    return corpus


In [15]:
# train_corpus_sentence_split = load_corpus_sentence_splitter(TRAIN_FILES, verbose=True)
val_corpus_sentence_split = load_corpus_sentence_splitter(VAL_FILES, verbose=True)

Loading files ['/content/data/batch0preprocessed_file.txt']
Loaded 1 docs


Parsing nodes:   0%|          | 0/1 [00:00<?, ?it/s]

Parsed 52 nodes


We do a very naive train/val split by having the
.


In [18]:
# with open(TRAIN_CORPUS_FPATH_SENTENCE_SPLIT, 'w+') as f:
#     json.dump(train_corpus_sentence_split, f)

with open(EVAL_CORPUS_FPATH_SENTENCE_SPLIT, 'w+') as f:
    json.dump(val_corpus_sentence_split, f)

## importings and inporting data

#  Initilazing HuggingFace LLM

In [5]:
!pip install llama-index-llms-huggingface -q -U

In [2]:
%pip install -q -U git+https://github.com/huggingface/transformers.git
%pip install accelerate -q -U
%pip install -i https://pypi.org/simple/ bitsandbytes -q
%pip install sentence-transformers -q
%pip install --upgrade transformers -q -U
%pip install llama-index-llms-openai -q -U
%pip install llama-index-embeddings-openai -q -U
%pip install llama-index-finetuning



  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.4/15.4 MB 50.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 22.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 326.8/326.8 kB 43.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 60.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 397.7/397.7 kB 29.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 173.8

In [6]:
!pip install --upgrade transformers -q


In [32]:
!pip install transformers[torch] -q


In [3]:
!pip install accelerate -q -U

## quantization Configuration and initializing LLM zephyr-7b-beta

In [7]:
import torch
from transformers import BitsAndBytesConfig
from llama_index.core.prompts import PromptTemplate
from llama_index.llms.huggingface import HuggingFaceLLM


import os
import warnings

# warnings.filterwarnings('ignore')

/usr/local/lib/python3.10/dist-packages/pydantic/_internal/_fields.py:160: UserWarning: Field "model_id" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


In [8]:
# use this code when using GPU
quantization_conf = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)


# # Disable GPU-specific configurations
# quantization_conf = BitsAndBytesConfig(
#     load_in_4bit=False,  # Disable 4-bit quantization
#     bnb_4bit_compute_dtype=torch.float32,  # Use float32 for CPU
#     # bnb_4bit_quant_type= None,  # Disable quantization type
#     bnb_4bit_use_double_quant=False  # Disable double quantization
# )




#We recreate template (or messages) and ensure that they have the correct format, as per,
#https://github.com/run-llama/llama_index/issues/9277#issuecomment-1837545398 for zephyr-7b-beta

def messages_to_prompt(messages):
    prompt = ""
    for message in messages:
        if message.role == 'system':
            prompt += f"<|system|>\n{message.content}</s>\n"
        elif message.role == 'user':
            prompt += f"<|user|>\n{message.content}</s>\n"
        elif message.role == 'assistant':
            prompt += f"<|assistant|>\n{message.content}</s>\n"

    # ensure we start with a system prompt, insert blank if needed
    if not prompt.startswith("<|system|>\n"):
        prompt = "<|system|>\n</s>\n" + prompt

    # add final assistant prompt
    prompt = prompt + "<|assistant|>\n"
    return prompt

### for GPU settings

In [9]:
def huggingface_llm(model_name="HuggingFaceH4/zephyr-7b-beta",
                    tokenizer_name="HuggingFaceH4/zephyr-7b-beta",
                    context_window=3900,
                    max_new_tokens=256,
                    quantization_config = quantization_conf
                   ):
    llm = HuggingFaceLLM(
        model_name=model_name,
        tokenizer_name=tokenizer_name,
        query_wrapper_prompt=PromptTemplate("<|system|>\n</s>\n<|user|>\n{query_str}</s>\n<|assistant|>\n"),
        context_window=context_window,
        max_new_tokens=max_new_tokens,
        model_kwargs={"quantization_config": quantization_config},
        # tokenizer_kwargs={},
        generate_kwargs={"temperature": 0.7, "top_k": 50, "top_p": 0.95},
        messages_to_prompt=messages_to_prompt,
        device_map="auto",
    )

    return llm

llm = huggingface_llm()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/638 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00008.safetensors:   0%|          | 0.00/1.89G [00:00<?, ?B/s]

model-00002-of-00008.safetensors:   0%|          | 0.00/1.95G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.43k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/168 [00:00<?, ?B/s]

### LLM  with CPU Setting

In [ ]:
%pip install llama-index-llms-huggingface-api -q -U


In [ ]:
def huggingface_llm(model_name="stabilityai/stable-code-3b",
                    tokenizer_name="stabilityai/stable-code-3b",
                    context_window=3900,
                    max_new_tokens=256,
                    # quantization_config = quantization_conf
                   ):
    llm = HuggingFaceLLM(
        model_name=model_name,
        tokenizer_name=tokenizer_name,
        query_wrapper_prompt=PromptTemplate("<|system|>\n</s>\n<|user|>\n{query_str}</s>\n<|assistant|>\n"),
        context_window=context_window,
        max_new_tokens=max_new_tokens,
        # model_kwargs={"quantization_config": quantization_config},
        # tokenizer_kwargs={},
        generate_kwargs={ "top_k": 50, do_sample=True},
        messages_to_prompt=messages_to_prompt,
        device_map="cpu",
    )

    return llm

llm = huggingface_llm()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


# Generate synthetic data


## Generating QA Embedding Pair

In [ ]:
TRAIN_QUERIES_FPATH = './data/train_queries.json'
TRAIN_RELEVANT_DOCS_FPATH = './data/train_relevant_docs.json'

VAL_QUERIES_FPATH = './data/val_queries.json'
VAL_RELEVANT_DOCS_FPATH = './data/val_relevant_docs.json'

In [ ]:
# with open(TRAIN_CORPUS_FPATH_SENTENCE_SPLIT, 'r+') as f:
#     train_corpus = json.load(f)

with open(EVAL_CORPUS_FPATH_SENTENCE_SPLIT, 'r+') as f:
    val_corpus = json.load(f)

In [21]:
from typing import Dict, List, Tuple

from llama_index.core.bridge.pydantic import BaseModel
from llama_index.core.llms.utils import LLM
from llama_index.core.schema import MetadataMode, TextNode
from tqdm import tqdm


In [22]:
class EmbeddingQAFinetuneDataset(BaseModel):
    """Embedding QA Finetuning Dataset.

    Args:
        queries (Dict[str, str]): Dict id -> query.
        corpus (Dict[str, str]): Dict id -> string.
        relevant_docs (Dict[str, List[str]]): Dict query id -> list of doc ids.

    """

    queries: Dict[str, str]  # dict id -> query
    corpus: Dict[str, str]  # dict id -> string
    relevant_docs: Dict[str, List[str]]  # query id -> list of doc ids
    mode: str = "text"

    @property
    def query_docid_pairs(self) -> List[Tuple[str, List[str]]]:
        """Get query, relevant doc ids."""
        return [
            (query, self.relevant_docs[query_id])
            for query_id, query in self.queries.items()
        ]

    def save_json(self, path: str) -> None:
        """Save json."""
        with open(path, "w") as f:
            json.dump(self.dict(), f, indent=4)

    @classmethod
    def from_json(cls, path: str) -> "EmbeddingQAFinetuneDataset":
        """Load json."""
        with open(path) as f:
            data = json.load(f)
        return cls(**data)


DEFAULT_QA_GENERATE_PROMPT_TMPL = """\
Context information is below.

---------------------
{context_str}
---------------------

Given the context information and no prior knowledge.
generate only questions based on the below query.

You are a Teacher/ Professor. Your task is to setup \
{num_questions_per_chunk} questions for an upcoming \
quiz/examination. The questions should be diverse in nature \
across the document. Restrict the questions to the \
context information provided."
"""
# generate queries as a convenience function

def generate_qa_embedding_pairs(
    node_dict,
    llm: LLM,
    qa_generate_prompt_tmpl: str = DEFAULT_QA_GENERATE_PROMPT_TMPL,
    num_questions_per_chunk: int = 2,
) -> EmbeddingQAFinetuneDataset:
    """Generate examples given a set of nodes."""


    queries = {}
    relevant_docs = {}
    for node_id, text in tqdm(node_dict.items()):
        query = qa_generate_prompt_tmpl.format(
            context_str=text, num_questions_per_chunk=num_questions_per_chunk
        )
        response = llm.complete(query)

        result = str(response).strip().split("\n")
        questions = [
            re.sub(r"^\d+[\).\s]", "", question).strip() for question in result
        ]
        questions = [question for question in questions if len(question) > 0][
            :num_questions_per_chunk
        ]

        num_questions_generated = len(questions)
        if num_questions_generated < num_questions_per_chunk:
            warnings.warn(
                f"Fewer questions generated ({num_questions_generated}) "
                f"than requested ({num_questions_per_chunk})."
            )

        for question in questions:
            question_id = str(uuid.uuid4())
            queries[question_id] = question
            relevant_docs[question_id] = [node_id]

        print(questions)

    # construct dataset
    return EmbeddingQAFinetuneDataset(
        queries=queries, corpus=node_dict, relevant_docs=relevant_docs
    )

In [ ]:
train_dataset = generate_qa_embedding_pairs(train_corpus, llm)

100%|██████████| 119/119 [28:05<00:00, 14.17s/it]


In [ ]:
train_dataset.save_json("trained_synthetic_dataset.json")

In [ ]:
train_synthetic_dataset = EmbeddingQAFinetuneDataset.from_json("/content/trained_synthetic_dataset.json")

In [23]:
# generating evaluation synthetic data set
eval_dataset = generate_qa_embedding_pairs(val_corpus, llm)

  0%|          | 0/52 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:540: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:545: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
  2%|▏         | 1/52 [00:29<25:29, 29.99s/it]Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


['What is the purpose of the Ordinance presented in the context information, and what actions does it propose to take?', 'Who are the sponsors of this Ordinance, and what is the proposed timeline for its implementation?']


  4%|▍         | 2/52 [00:43<16:42, 20.05s/it]Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


['What is the purpose of Ordinance No. 153-22, and what actions does it propose to take in relation to the County Transportation Authority and sales tax?', "How does the proposed Ordinance amend the Business and Tax Regulations Code, and what is the duration and rate of the local transactions and use tax that it seeks to continue in effect? Additionally, what is the amount by which the Transportation Authority's appropriations limit would be increased, and for how long? Finally, what is the role of limited tax bonds in this proposal, and what is the basis for the Transportation Authority's determination under the California Environmental Quality Act?"]


  6%|▌         | 3/52 [01:05<17:25, 21.35s/it]Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


...

["What is the purpose of the Board of Supervisors' decision to adopt, establish, and designate voting precincts, polling places, and officers of election for the Special Election called?", 'What is the role of the Department of Elections in relation to the Special Election called by the Board of Supervisors, as directed by California Public Utilities Code Section 131108, subdivision (h)?']


 98%|█████████▊| 51/52 [14:17<00:14, 14.13s/it]Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


['Based on the context information provided, what is the role of the Supervisors Mandelman, Walton, Peskin, Ronen, Melgar, Stefani, Preston, Mar, and Safai in this situation?', 'What is the significance of California Public Utilities Code Section 131055 mentioned in the context information? What action is being directed by the Board of Supervisors in relation to this section?']


100%|██████████| 52/52 [14:25<00:00, 16.64s/it]

['What is the purpose of the Ordinance passed by the Board of Supervisors on July 19, 2022, and what does it entail?', 'What is the California Environmental Quality Act, and why is it mentioned in the context information provided? Does the Ordinance mentioned in the context information require any environmental assessments or approvals? If so, what are they?']


In [24]:
eval_dataset.save_json("eval_synthetic_qa_dataset.json")

## Generating Anchor and positive pair

In [ ]:
class EmbeddingAnchorPositiveDataset(BaseModel):
    """Embedding Anchor-Positive Dataset.

    Args:
        anchors (Dict[str, str]): Dict id -> anchor text.
        positives (Dict[str, str]): Dict id -> positive text.
        similarities (Dict[str, float]): Dict id -> similarity score.
        corpus (Dict[str, str]): Dict id -> string.
    """

    anchors: Dict[str, str]  # dict id -> anchor text
    positives: Dict[str, str]  # dict id -> positive text
    similarities: Dict[str, float]  # dict id -> similarity score
    corpus: Dict[str, str]  # dict id -> string
    mode: str = "text"

    @property
    def anchor_positive_pairs(self) -> List[Tuple[str, str, float]]:
        """Get anchor, positive pairs, and their similarity scores."""
        return [
            (self.anchors[anchor_id], self.positives[anchor_id], self.similarities[anchor_id])
            for anchor_id in self.anchors.keys()
        ]

    def save_json(self, path: str) -> None:
        """Save json."""
        with open(path, "w") as f:
            json.dump(self.dict(), f, indent=4)

    @classmethod
    def from_json(cls, path: str) -> "EmbeddingAnchorPositiveDataset":
        """Load json."""
        with open(path) as f:
            data = json.load(f)
        return cls(**data)


In [ ]:
DEFAULT_ANCHOR_POSITIVE_PROMPT_TMPL = """\
Context information is below.

---------------------
{context_str}
---------------------

Given the context information and no prior knowledge,
generate anchor and positive pairs based on the below query.

You are an NLP specialist. Your task is to identify \
{num_pairs_per_chunk} anchor-positive pairs for semantic similarity. \
For each pair, also provide a similarity score as a float point value. \
The similarity score should be between 0 and 1, indicating how similar \
the anchor is to the positive example. Restrict the pairs to the \
context information provided.

Format:
Anchor | Positive | Similarity
------------------------------
"""

def generate_anchor_positive_pairs(
    node_dict,
    llm: LLM,
    prompt_tmpl: str = DEFAULT_ANCHOR_POSITIVE_PROMPT_TMPL,
    num_pairs_per_chunk: int = 2,
) -> EmbeddingAnchorPositiveDataset:
    """Generate anchor-positive pairs given a set of nodes."""

    anchors = {}
    positives = {}
    similarities = {}

    for node_id, text in tqdm(node_dict.items()):
        print(node_id)
        print(text)
        query = prompt_tmpl.format(
            context_str=text, num_pairs_per_chunk=num_pairs_per_chunk
        )
        response = llm.complete(query)

        print("response:  ->", resonse)
        result = str(response).strip().split("\n")
        print("results:  ->", results)
        pairs = [
            line.split(" | ") for line in result if "|" in line
        ]
        pairs = [
            (anchor.strip(), positive.strip(), float(similarity.strip()))
            for anchor, positive, similarity in pairs
        ]

        num_pairs_generated = len(pairs)
        if num_pairs_generated < num_pairs_per_chunk:
            warnings.warn(
                f"Fewer pairs generated ({num_pairs_generated}) "
                f"than requested ({num_pairs_per_chunk})."
            )

        for anchor, positive, similarity in pairs:
            pair_id = str(uuid.uuid4())
            anchors[pair_id] = anchor
            positives[pair_id] = positive
            similarities[pair_id] = similarity

        print(pairs)

    # construct dataset
    return EmbeddingAnchorPositiveDataset(
        anchors=anchors, positives=positives, similarities=similarities, corpus=node_dict
    )



NameError: name 'response' is not defined

In [ ]:
# Example usage
train_corpus = {
    "1": "the quick brown fox jumps over the lazy dog.",
    # "2": "The Way It Is Now\nCHANGES TO BUSINESS TAXES\nThe City collects various business taxes on an annual basis including:\nO\n\u2022\nSAN FRANCISCO\nFILED\n2024 MAY 15 PM 3:10\nDEPARTMENT OF ELECTIONS\nA gross receipts tax that is a percentage of a business's San Francisco gross receipts.\nDepending on business type, the City determines a business's San Francisco gross\nreceipts based on sales in San Francisco, payroll expenses for employees working there,\nor both. Rates range from 0.053% to 1.008% and are scheduled to increase in coming\nyears. Rates depend on business type, and higher rates apply as a business generates\nmore gross receipts. For 2023, most businesses with gross receipts up to $2.19 million\nare exempt.\nA homelessness gross receipts tax that is an additional tax on businesses with San\nFrancisco gross receipts over $50 million. Rates range from 0.175% to 0.69%.\nAn overpaid executive gross receipts tax that is an additional tax on businesses that pay\ntheir highest-paid managerial employee much higher than the median compensation they\npay their San Francisco employees. Rates are between 0.1% and 0.6%.\nA business registration fee that is an additional tax. For most businesses the fee is\ncurrently between $47 and $45,150, based on business type and amount of gross receipts.\n\u2022 An administrative office tax on payroll expenses that certain large businesses pay instead\nof these other business taxes. The combined rates in 2024 range from 3.04% to 5.44%,\nand in 2025 are scheduled to range from 3.11% to 5.51%. Business registration fees for\nthese businesses currently range from $19,682 to $45,928.\nState law limits the total revenue, including tax revenue, the City may spend each year. The\nvoters may approve increases to this limit for up to four years.",
    # "3": "Punjab, known as the 'Land of Five Rivers,' is a vibrant Indian state renowned for its rich cultural heritage, delectable cuisine, colorful festivals, and warm hospitality"
}


In [ ]:


train_dataset = generate_anchor_positive_pairs(train_corpus, llm)

# Save the dataset to a JSON file
train_dataset.save_json("train_dataset.json")

# Load the dataset from a JSON file
loaded_dataset = EmbeddingAnchorPositiveDataset.from_json("train_dataset.json")


  0%|          | 0/1 [00:00<?, ?it/s]Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


1
he quick brown fox jumps over the lazy dog.


  0%|          | 0/1 [04:59<?, ?it/s]


NameError: name 'resonse' is not defined

# Embedding Finetuning


## checking generated data of qa pairs

In [5]:
train_synthetic_dataset = EmbeddingQAFinetuneDataset.from_json("/content/trained_synthetic_dataset.json")

In [6]:
train_synthetic_dataset.queries["a0bce105-3c69-47a0-a3ff-fc5df8c181e8"]

'What is the role of the Controller in determining the applicable tax rates for the gross receipts tax in Santa Cruz?'

In [ ]:
train_synthetic_dataset.corpus["36b77480-727d-40f4-8490-5e845e808178"]


'CONTROLLER TO PUBLISH AND CERTIFY TAXABLE GROSS RECEIPTS\nAMOUNTS.\n(a) On or before October 3, 2022, for purposes of determining the applicable tax rates for tax\nyear 2023, the Controller shall publish the total amount of taxable gross receipts for tax year 2021\nreported by taxpayers as of June 30, 2022, and if that amount is equal to or greater than 90% of\n53SANTA CO\ntaxable gross receipts for tax year 2019 reported by taxpayers as of June 30, 2020, the Controller shall\nDEPARTMEN OP QRECTIONS\ncertify that the 90% gross receipts threshold has been met for tax year 2023.\n(b) On or before October 2, 2023, for purposes of determining the applicable tax rates for tax\nyear 2024, the Controller shall publish the total amount of taxable gross receipts for tax year 2022\nreported by taxpayers as of June 30, 2023, and if that amount is equal to or greater than 95% of\ntaxable gross receipts for tax year 2019 reported by taxpayers as of June 30, 2020, the Controller shall\neertify that

## imports

In [32]:
!pip install -U sentence-transformers -q

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llama-index-finetuning 0.1.9 requires sentence-transformers<3.0.0,>=2.3.0, but you have sentence-transformers 3.0.1 which is incompatible.


In [33]:
from sentence_transformers import SentenceTransformer

In [34]:
model_id = "BAAI/bge-small-en"
model = SentenceTransformer(model_id)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/90.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/684 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
model

SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': True}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': True, 'pooling_mode_mean_tokens': False, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
)

In [25]:
import json

from torch.utils.data import DataLoader
from sentence_transformers import InputExample

In [48]:
TRAIN_DATASET_FPATH = '/content/trained_synthetic_qa_dataset.json'
VAL_DATASET_FPATH = '/content/eval_synthetic_qa_dataset.json'


BATCH_SIZE = 10

In [49]:
with open(TRAIN_DATASET_FPATH, 'r+') as f:
    train_dataset = json.load(f)

with open(VAL_DATASET_FPATH, 'r+') as f:
    val_dataset = json.load(f)

In [50]:
dataset = train_dataset

corpus = dataset['corpus']
queries = dataset['queries']
relevant_docs = dataset['relevant_docs']

examples = []
for query_id, query in queries.items():
    node_id = relevant_docs[query_id][0]
    text = corpus[node_id]
    example = InputExample(texts=[query, text])
    examples.append(example)


In [51]:
loader = DataLoader(
    examples, batch_size=BATCH_SIZE
)

## MultipleNegativesRankingLoss

### Define loss
MultipleNegativesRankingLoss is a great loss function if you only have positive pairs, for example, only pairs of similar texts like pairs of paraphrases, pairs of duplicate questions, pairs of (query, response), or pairs of (source_language, target_language).

This loss function works great to train embeddings for retrieval setups where you have positive pairs (e.g. (query, relevant_doc)) as it will sample in each batch n-1 negative docs randomly.


The performance usually increases with increasing batch sizes.


In [30]:
from sentence_transformers import losses

In [35]:
loss = losses.MultipleNegativesRankingLoss(model)


### Defing Evaluator

We setup an evaluator with our val split of the dataset to monitor how well the embedding model is performing during training.



In [26]:
from sentence_transformers.evaluation import InformationRetrievalEvaluator


In [61]:
dataset = val_dataset

corpus = dataset['corpus']
queries = dataset['queries']
relevant_docs = dataset['relevant_docs']

evaluator = InformationRetrievalEvaluator(queries, corpus, relevant_docs,write_csv=True)

### Run Training
The training loop is very straight forward to steup thanks to sentencetransformers' high-level model training API. All we need to do is plugging in the data loader, loss function, and evaluator that we defined in the previous cells (along with a couple of additional minor settings).

In [38]:
!pip3 install datasets -q -U


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 24.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 16.1.0 which is incompatible.
google-colab 1.0.0 requires requests==2.31.0, but you have requests 2.32.3 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 16.1.0 which is incompatible.
llama-index-finetuning 0.1.9 requ

In [38]:
!pip install --upgrade pyarrow datasets -q

In [41]:
!pip uninstall pyarrow datasets -y
!pip install pyarrow==14.0.1 datasets -q


Found existing installation: pyarrow 16.1.0
Uninstalling pyarrow-16.1.0:
  Successfully uninstalled pyarrow-16.1.0
Found existing installation: datasets 2.20.0
Uninstalling datasets-2.20.0:
  Successfully uninstalled datasets-2.20.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.0/38.0 MB 21.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.1/542.1 kB 54.4 MB/s eta 0:00:00


In [43]:
!pip install accelerate -U -q

In [22]:
!pip install transformers[torch] -q

In [ ]:
from datasets import Dataset

In [72]:
# We train the model for very few epochs in this toy example.
# This should typically be higher for better performance.
EPOCHS = 1

In [68]:
warmup_steps = int(len(loader) * EPOCHS)

In [73]:
model.fit(
    train_objectives=[(loader, loss)],
    epochs=EPOCHS,
    warmup_steps=warmup_steps,
    output_path='exp_finetune',
    show_progress_bar=True,
    evaluator=evaluator,
    # evaluation_steps=50,
)

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/24 [00:00<?, ?it/s]

### checking the evaluation

In [74]:
results = evaluator(model)

In [76]:
print(results)

0.7249176564282218


## Sharing a Sentence Transformers to the Hugging Face Hub

In [59]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): Traceback (most recent call last):
  File "/usr/lib/python3.10/getpass.py", line 77, in unix_getpass
    passwd = _raw_input(prompt, stream, input=input)
  File "/usr/lib/python3.10/getpass.py", line 146, in _raw_input
    line = input.readline()
KeyboardInterrupt

During han

In [ ]:
# 6. Save the trained model and optionally push it to the Hugging Face Hub
model.save_pretrained("bert-base-all-nli-stsb-quora-nq")
model.push_to_hub("bert-base-all-nli-stsb-quora-nq")

In [1]:
# Define the directory to save the model
save_path = 'fine_tuned_model'

# Save the model
model.save(save_path)

NameError: name 'model' is not defined

In [79]:
import shutil
from google.colab import files

# Zip the model directory
shutil.make_archive(save_path, 'zip', save_path)

# Download the zip file
files.download(f'{save_path}.zip')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

##